## 2. Kalman Filter

The Extended Kalman Filter, an extension of the classic Kalman Filter for nonlinear systems, is an ideal tool for this scenario. It helps in predicting the future state of the robot by estimating its current position and movement, even in the absence of direct positional inputs from the external camera. 

#### Initial Approach

Initially, our approach to state estimation in the Thymio robot navigation project relied on using motor speed and camera inputs as the primary data sources for the Kalman filter. This setup seemed promising as it directly utilized the robot's inherent capabilities to infer its movement and orientation.


#### Problem Encountered

However, we soon encountered a significant challenge. The speed sensor in the Thymio robot exhibited a high degree of noise. This noise severely distorted the state estimation. For example, at a true speed of 50 mm/s, our estimates varied wildly between 20 and 80 mm/s. Such a wide margin of error was unacceptable for precise navigation and pose estimation.


#### Solution: State Space Equations for Differential Drive Robot

To address this, we shifted our focus to a more robust solution. We decided to predict the state of the robot based on the state space equations of a differential drive robot. This method relies less on noisy sensor data and more on the physical and theoretical model of the robot's movement.

In [9]:
class extended_kalman:
	def __init__(self, Q, H, R, sampling_rate):
		self._Q = Q
		self._H = H
		self._R = R
		self._sampling_rate = sampling_rate        

	def predict(self, x_init, P_init, u):
		F = np.eye(3) #+ np.array([[0, 0, -u[0]*np.sin(x_init[2])], [0, 0, u[0]*np.cos(x_init[2])], [0, 0, 0]])*self._sampling_rate
		G = np.array([[np.cos(x_init[2]), 0], [np.sin(x_init[2]), 0], [0, -1]])*self._sampling_rate
		x_est = np.dot(F, x_init) + np.dot(G, u)
		P_est = np.dot(np.dot(F, P_init), F.T) + self._Q
		
		return x_est, P_est
	
	def correct(self, x_est, P_est, y):
		i = y - np.dot(self._H, x_est)
		j = np.eye(len(self._H))
		S = np.dot(self._H, np.dot(P_est, self._H.T)) + self._R
		Kn = np.dot(np.dot(P_est, self._H.T), np.linalg.inv(S))
		
		x_final = x_est + np.dot(Kn, i)
		P_final = np.dot(j, np.dot(P_est, j.T)) + np.dot(Kn, np.dot(self._R, Kn.T))
		
		return x_final, P_final
	

	def run(self, x_init, P_init, u, y):
		x_est, P_est = self.predict(x_init, P_init, u)
		return self.correct(x_est, P_est, y)

#### Initialization

The constructor __init__ initializes the filter with necessary parameters:

- `Q` : Process noise covariance matrix
- `H` : Measurement matrix
- `R` : Measurement noise covariance matrix
- `sampling_rate` : The rate at which the measurements are sampled

#### Prediction Step

The predict method implements the prediction step of the EKF.

- `x_init`: Initial state estimate
- `P_init` : Initial covariance estimate
- `u` : Control input vector
In this step, we calculate the estimated state x_est and the covariance of the estimate P_est using the state transition model and control input.

#### Correction Step

The correct method implements the correction step.

- `x_est`: Estimated state from the prediction step
- `P_est`: Estimated covariance from the prediction step
- `y` : Measurement vector

This step updates the estimated state and covariance based on the measurement y.

#### Running the Filter
The run method combines the predict and correct steps to process a single measurement.

`x_init`, `P_init`, `u`, `y`: Inputs for the prediction and correction steps

## 3. Path Finding

The goal of this section is to find the shortest path between two points in a grid-like structure while minimizing the total cost associated with the path. <br /><br />
We chose to use the A* Algorithm because it it garanteed to find the shortest path and it is more efficient than other algorithms in terms of both time and memory usage.<br/><br />
We use this algorithm to find the shortest path from the robot initial position to the robot targeted position. At this point of the project we are taking into account only the global obstacles visible from the camera, and not the local ones.


*<u>Image 3.1:</u>* _Example of the shortest path from the Thymio robot (top left) to the target (bottom right) using A* Algorithm._

![a-star-path](images/a-star-path-example.jpg)


The Node class initializes each node with its coordinates (coord) and the goal node's coordinates (goal).
- The g value is the cost from the start node to the current node, initialized to infinity.
- The _h value is the heuristic estimate  of the cost to reach the goal from the current node.
- The _f value is the sum of g and _h, used to determine the node's priority in the pathfinding process.
- The set_cost and set_parent methods are used to update a node's cost and its parent in the path.
- The _get_dist method calculates the heuristic distance (Manhattan distance) between two nodes, considering both straight and diagonal movements.

The A_star class is initialized with a grid and a set of camera coordinates.
- We chose 8 possible movements directions from a node.
- The _path_constructor method reconstructs the path from the start to the goal node, traversing backward from the goal.
- The _is_traversible method checks if a coordinate is within the grid and thus accessible.
- The find_path method implements the core A* algorithm. It uses a priority queue (heapq) to efficiently select the next node to explore. The method returns the constructed path from start to goal if available, or an empty list if no path is found.

In [3]:
from PathFinding import Node

node = Node((2, 0), (20,20))
print(node.coord)

(2, 0)


The set_motors function is designed to control the two-wheeled Thymio robot by adjusting the speed of its left and right motors based on specific parameters. 

Function Definition: def set_motors(th, v=0, w=0):

v: Linear velocity (in mm/s). It's the forward speed at which the robot moves. Default is 0.
w: Angular velocity (in rad/s). It represents the rate of rotation of the robot. Default is 0.

- Conversion Factor: conv_factor = 450/140

This factor is used to convert the calculated wheel speeds to a value that is compatible with the motor's input. The exact values (450 and 140) are specific to the robot's design and motor specifications.

- Robot's Wheel Distance: L = 95 (in mm)

This represents the distance between the centers of the two wheels of the robot. It's crucial for calculating the turning effect.

- Calculating Wheel Speeds:

The function calculates the target speeds for the right (speedR) and left (speedL) wheels.
The formula `(2*v + w*L)/2` and `(2*v - w*L)/2` are derived from the differential drive kinematics. Here, v is the linear velocity and `w*L` represents the rotational component. The right wheel speed is increased by `w*L` for a right turn and decreased for a left turn, and vice versa for the left wheel.
Applying Conversion Factor:



<div style="text-align: center;">
  <img src="images\robot-pd-path.png"alt="Alt Text" id="img_4.0" width="700">
  <p style="text-align: center;">Image 4.0: A* computed path (green) vs Controller path (red). </p>
</div>